In [1]:
import pandas as pd

optimizers = [
    "adadelta",
    "adagrad",
    "adam",
    "adamax",
    "adamw",
    "asgd",
    "lbfgs",
    "nadam",
    "radam",
    "rmsprop",
    "rprop",
    "sgd",
    "sparseadam"
]

datasets = [
    "Cora",
    "BZR"
]

In [2]:
mean = None
for opt in optimizers:
    df = pd.read_csv(f"results_{opt}.csv", index_col='index')
    df['optimizer'] = opt
    df
    mean = pd.concat([mean, df], axis=0)

mean = mean.drop(index=0)
mean = mean.drop(labels=["K", "epochs_number"], axis=1)

# display(data)
# df.index.nunique()

In [3]:
grouped = mean.groupby(by=["dataset", "layer", "layers_number", "optimizer"])
mean = grouped.mean()
var = grouped.var()
mean.reset_index(inplace=True)
var.reset_index(inplace=True)

In [18]:
mean

,dataset,layer,layers_number,optimizer,training_time,train_accuracy,train_F1,test_accuracy,test_F1
0,AIDS,GATConv,2,adam,101.400739,0.930875,0.886738,0.933500,0.887475
1,AIDS,GATConv,3,adam,94.145846,0.980625,0.968835,0.984500,0.975912
2,AIDS,GATConv,4,adam,81.553242,0.982625,0.972515,0.982000,0.970486
3,AIDS,GCNConv,2,adam,157.124819,0.949625,0.916105,0.945000,0.913915
4,AIDS,GCNConv,3,adam,243.277059,0.990875,0.985735,0.990500,0.984095
...,...,...,...,...,...,...,...,...,...
665,PubMed,SGConv,2,adam,76.910985,0.849033,0.843966,0.846400,0.841358
666,PubMed,SGConv,3,adam,84.445570,0.863691,0.855708,0.857809,0.849855
667,PubMed,SSGConv,1,adam,166.228989,0.787206,0.775857,0.783722,0.772547
668,PubMed,SSGConv,2,adam,104.624854,0.857643,0.852979,0.856542,0.852944


table accuracy
        1xgcn   2xgcn ...
adam
adamw
sgd
...

In [9]:
# field = "test_accuracy"
field = "training_time"
dataset = "BZR"
t_optims = optimizers
t_layers = ["GCNConv"]
filename = f"tables/optim_{field}_{dataset}.txt"

l_nums_by_dataset = {"Cora": [1, 2, 3], "BZR": [2, 3, 4]}
l_nums = l_nums_by_dataset[dataset]

num_cols = len(l_nums) * len(t_layers) + 1

t_header = ("\\begin{table}[h!]\n"
            "\\centering\n"
            f"\\begin{{tabular}}{{{'c' * num_cols}}}\n")
t_footer = ("\\end{tabular}\n"
           f"\\caption{{Сравнение оптимайзеров. {field} {dataset}}}\n"
           "\\label{tab:my_label}\n"
           "\\end{table}\n")

with open(filename, "w") as f:
    f.write(t_header)
    f.write("Оптимайзер\t&\t")
    for l in t_layers:
        for n in l_nums:
            f.write(f"{n}x{l}\t")
            if l == t_layers[-1] and n == l_nums[-1]:
                f.write("\t\\\\\n")
            else:
                f.write("&\t")
    
    for o in t_optims:
        f.write(f"{o}\t&\t")
        for l in t_layers:
            for n in l_nums:
                mean_val = mean.query(f"optimizer == '{o}' & layer == '{l}' & layers_number == {n} & dataset == '{dataset}'")[field].values
                if len(mean_val) != 1:
                    continue
                mean_val = mean_val[0]

                var_val = var.query(f"optimizer == '{o}' & layer == '{l}' & layers_number == {n} & dataset == '{dataset}'")[field].values
                if len(var_val) != 1:
                    continue
                var_val = var_val[0]

                f.write(f"{mean_val:.2f}±{var_val ** 0.5:.2f}\t")
                if l == t_layers[-1] and n == l_nums[-1]:
                    f.write("\t\\\\\n")
                else:
                    f.write("&\t")

    f.write(t_footer)



In [ ]:
mean